In [13]:
! pip install pandas numpy scikit-learn nltk matplotlib seaborn wordcloud spacy flask openpyxl
! python -m nltk.downloader stopwords
! python -m spacy download en_core_web_sm

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from flask import Flask, request, jsonify

In [17]:
# Cargamos los datos 
ODS_Train_data = pd.read_csv('./dataset/ODScat_345.csv' , encoding='utf-8')

# 1. Entendimiento de los Datos

In [20]:
#Leemos el dataset ODScat_345 
ODS_train_data = pd.read_csv('./dataset/ODScat_345.csv')
ODS_train_data.head()

,Textos_espanol,sdg
0,"Por ejemplo, el nÃºmero de consultas externas ...",3
1,"En 2007, el gobierno central financió directam...",3
2,"Claramente, hay muchos otros factores en juego...",3
3,"Por ejemplo, el estado australiano de Victoria...",3
4,"El consumo anual de alcohol se estima en 15,7 ...",3


# 2. Preparación de datos 

In [21]:
ODS_train_data.shape

(4049, 2)

In [24]:
ODS_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4049 entries, 0 to 4048
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Textos_espanol  4049 non-null   object
 1   sdg             4049 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 63.4+ KB


# 3. Modelado y Evaluación

## 3.1 Creación de modelos

Modelo 1

In [ ]:
print("first model")

Modelo 2 

In [ ]:
print ("Second Model")

Modelo 3

In [ ]:
print("third model")

## 3.2 Evaluación de modelos 